In [13]:
from src.code.tools.metrics import calculate_metrics
import os
from src.code.TFIDF.pre_processing import Processing
from nltk.corpus import stopwords
from dotenv import load_dotenv
from src.code.graphe.build_graph import Graphe
from tqdm import tqdm
import glob
from nltk.tokenize import sent_tokenize
from src.code.bert.bert import Bert
load_dotenv()

True

In [15]:
metricis = {'Précision': 0,
            'Rappel': 0,
            'F-mesure': 0,
            'Spécificité': 0,
            'Taux de faux positifs': 0}
documents_count = 0
for file in tqdm(glob.glob(os.path.join(os.getenv('TXT_FOLDER2'), "*.txt"))[1:10]):
    with open(file, "r", encoding="utf-8") as f:
        doc = f.read().strip().lower()
        if len(doc) > 10:
            documents_count += 1
            documents = sent_tokenize(doc)
            bert = Bert(documents=documents)
            bert.processing()
            top_indices, summary = bert.get_resume(top_n=10)
            preprocessing = Processing(
                log_file="", corpus_dir="processed", documents=documents)
            preprocessing.process_corpus()
            resume_indeces, resume_docs = preprocessing.get_resume_docs(
                top_n=10, number_key_words=30)
            metricis_calcul = calculate_metrics(top_indices, resume_indeces)
            metricis['Précision'] += metricis_calcul['Précision']
            metricis['Rappel'] += metricis_calcul['Rappel']
            metricis['F-mesure'] += metricis_calcul['F-mesure']
            metricis['Spécificité'] += metricis_calcul['Spécificité']
metricis['Précision'] /= documents_count
metricis['Rappel'] /= documents_count
metricis['F-mesure'] /= documents_count
metricis['Spécificité'] /= documents_count
print(metricis)

100%|██████████| 9/9 [02:43<00:00, 18.13s/it]

{'Précision': 0.3444444444444444, 'Rappel': 0.3444444444444444, 'F-mesure': 0.34444444444444444, 'Spécificité': 0.0, 'Taux de faux positifs': 0}


In [12]:

calculate_metrics(top_indices, resume_indeces)

{'Précision': 0.5,
 'Rappel': 0.5,
 'F-mesure': 0.5,
 'Spécificité': 0.0,
 'Taux de faux positifs': 1.0}